In [7]:
#import all the essential libraries and packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
from scipy import stats
from scipy.stats import t, ttest_ind_from_stats
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

In [112]:
import requests
import sys
import json
from bs4 import BeautifulSoup
api_key = 'cf478da0d8784f26877dc94b06b0f8e8'
url = 'http://newsapi.org/v2/everything?'

parameters = {
    'q':'covid OR coronavirus OR pandemic',
    'apiKey':api_key,
    'language': 'en',
    'pageSize': 100,
    #'from':'2020-06-30'
}
response = requests.get(url, params=parameters)
response_json = response.json()
print(response_json)
print ("Estimated size: " + str(sys.getsizeof(response_json) / 1024) + "KB")


{'status': 'ok', 'totalResults': 431004, 'articles': [{'source': {'id': 'the-verge', 'name': 'The Verge'}, 'author': 'Adi Robertson', 'title': 'Whistleblower Reality Winner has tested positive for COVID-19 in prison', 'description': 'Former NSA contractor Reality Winner, who is imprisoned in Texas for leaking documents about Russian election interference, has reportedly been diagnosed with COVID-19 amid the coronavirus pandemic.', 'url': 'https://www.theverge.com/2020/7/20/21331782/reality-winner-covid-19-coronavirus-positive-diagnosis-fmc-carswell-prison', 'urlToImage': 'https://cdn.vox-cdn.com/thumbor/QdDFzxzpr8cNDBPRUIhqs4fWRTI=/0x0:4177x2187/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/20102785/693907654.jpg.jpg', 'publishedAt': '2020-07-20T21:56:56Z', 'content': 'She was denied release earlier in the pandemic\r\nPhoto by Sean Rayford / Getty Images\r\nFormer intelligence contractor and whistleblower Reality Winner has reportedly tested positive for COVID-19. Winn… [+2

In [67]:
#url = ('http://newsapi.org/v2/everything?'
#       'q=covid OR coronavirus OR pandemic&'
#       'language=en&'
#       'from=2020-06-30&'
#       'sortBy=popularity&'
#       'apiKey=cf478da0d8784f26877dc94b06b0f8e8'
#    )

#response = requests.get(url)

#print (response.json())
#print ("Estimated size: " + str(sys.getsizeof(response_json) / 1024) + "KB")

In [76]:
from newsapi import NewsApiClient

newsapi = NewsApiClient(api_key='cf478da0d8784f26877dc94b06b0f8e8')

all_articles = newsapi.get_everything(q='covid OR coronavirus OR pandemic',
                                      from_param='2020-06-30',
                                      to='2020-07-29',
                                      language='en',
                                      sort_by='relevancy',
                                      )
all_articles

{'status': 'ok',
 'totalResults': 397686,
 'articles': [{'source': {'id': 'the-verge', 'name': 'The Verge'},
   'author': 'Adi Robertson',
   'title': 'Whistleblower Reality Winner has tested positive for COVID-19 in prison',
   'description': 'Former NSA contractor Reality Winner, who is imprisoned in Texas for leaking documents about Russian election interference, has reportedly been diagnosed with COVID-19 amid the coronavirus pandemic.',
   'url': 'https://www.theverge.com/2020/7/20/21331782/reality-winner-covid-19-coronavirus-positive-diagnosis-fmc-carswell-prison',
   'urlToImage': 'https://cdn.vox-cdn.com/thumbor/QdDFzxzpr8cNDBPRUIhqs4fWRTI=/0x0:4177x2187/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/20102785/693907654.jpg.jpg',
   'publishedAt': '2020-07-20T21:56:56Z',
   'content': 'She was denied release earlier in the pandemic\r\nPhoto by Sean Rayford / Getty Images\r\nFormer intelligence contractor and whistleblower Reality Winner has reportedly tested positive 

In [113]:
response_json.keys() # we want the articles here to make our dataset

dict_keys(['status', 'totalResults', 'articles'])

In [114]:
covid_articles = response_json['articles'] # coverting json to readable format
covid_articles

[{'source': {'id': 'the-verge', 'name': 'The Verge'},
  'author': 'Adi Robertson',
  'title': 'Whistleblower Reality Winner has tested positive for COVID-19 in prison',
  'description': 'Former NSA contractor Reality Winner, who is imprisoned in Texas for leaking documents about Russian election interference, has reportedly been diagnosed with COVID-19 amid the coronavirus pandemic.',
  'url': 'https://www.theverge.com/2020/7/20/21331782/reality-winner-covid-19-coronavirus-positive-diagnosis-fmc-carswell-prison',
  'urlToImage': 'https://cdn.vox-cdn.com/thumbor/QdDFzxzpr8cNDBPRUIhqs4fWRTI=/0x0:4177x2187/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/20102785/693907654.jpg.jpg',
  'publishedAt': '2020-07-20T21:56:56Z',
  'content': 'She was denied release earlier in the pandemic\r\nPhoto by Sean Rayford / Getty Images\r\nFormer intelligence contractor and whistleblower Reality Winner has reportedly tested positive for COVID-19. Winn… [+2100 chars]'},
 {'source': {'id': 'techc

In [115]:
#function to extract data from json list of lists

def get_articles(file):
    extracted_articles = []
    
    for i in range(len(file)):
        article_dict = {} # dict to make key value pairs of all headings of articles
        article_dict['source'] = file[i]['source']
        article_dict['author'] = file[i]['author']
        article_dict['title'] = file[i]['title']
        article_dict['description'] = file[i]['description']
        article_dict['url'] = file[i]["url"]
        article_dict['photo_url'] = file[i]['urlToImage']
        article_dict['published_date'] = file[i]['publishedAt']
        article_dict['content'] = file[i]['content']
        
        extracted_articles.append(article_dict)
    return extracted_articles        

In [116]:
covid_articles_df = pd.DataFrame(get_articles(covid_articles))

In [117]:
covid_articles_df.shape

(100, 8)

In [110]:
covid_articles_df

,source,author,title,description,url,photo_url,published_date,content
0,"{'id': 'the-verge', 'name': 'The Verge'}",Adi Robertson,Whistleblower Reality Winner has tested positi...,"Former NSA contractor Reality Winner, who is i...",https://www.theverge.com/2020/7/20/21331782/re...,https://cdn.vox-cdn.com/thumbor/QdDFzxzpr8cNDB...,2020-07-20T21:56:56Z,She was denied release earlier in the pandemic...
1,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Lucas Matney,Apple smashes Q3 revenue expectations despite ...,COVID-19 isn’t keeping Apple down. The company...,http://techcrunch.com/2020/07/30/apple-smashes...,https://techcrunch.com/wp-content/uploads/2020...,2020-07-30T20:43:08Z,COVID-19 isn’t keeping Apple down.\r\nThe comp...
2,"{'id': None, 'name': 'Lifehacker.com'}",Elizabeth Yuko,How to Communicate with COVID Conspiracy Theor...,"Thanks in no small part to COVID-19, we have f...",https://lifehacker.com/how-to-communicate-with...,https://i.kinja-img.com/gawker-media/image/upl...,2020-07-21T20:30:00Z,"Thanks in no small part to COVID-19, we have f..."
3,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Brian Heater,"As pandemic drags on, interest in automation s...","Today, the U.S. exceeded three million COVID-1...",http://techcrunch.com/2020/07/09/as-pandemic-d...,https://techcrunch.com/wp-content/uploads/2020...,2020-07-09T15:30:12Z,"Today, the U.S. exceeded three million COVID-1..."
4,"{'id': 'bbc-news', 'name': 'BBC News'}",https://www.facebook.com/bbcnews,Coronavirus: UN makes record $10.3M appeal for...,The UN says up to 265 million people could fac...,https://www.bbc.co.uk/news/world-53439535,https://ichef.bbci.co.uk/news/1024/branded_new...,2020-07-17T01:13:57Z,Image copyrightGetty ImagesImage caption\r\n I...
...,...,...,...,...,...,...,...,...
95,"{'id': None, 'name': 'Lifehacker.com'}",Elizabeth Yuko,Build Your TBR With Free Books and Magazines T...,"Like it or not, Amazon Prime has become a majo...",https://lifehacker.com/build-your-tbr-with-fre...,https://i.kinja-img.com/gawker-media/image/upl...,2020-07-27T17:00:00Z,"Like it or not, Amazon Prime has become a majo..."
96,"{'id': 'wired', 'name': 'Wired'}",Steven Levy,The CEO of Novartis on Developing Drugs During...,"Vas Narasimhan talks about drug prices, vaccin...",https://www.wired.com/story/the-ceo-of-novarti...,https://media.wired.com/photos/5efa39b545997e7...,2020-06-30T12:00:00Z,"As CEO of Novartis, the worlds second-largest ..."
97,"{'id': None, 'name': 'Gizmodo.com'}",Matt Novak,Covid-19 Test Results Frequently Taking Over a...,"Quest Diagnostics, the largest private medical...",https://gizmodo.com/covid-19-test-results-freq...,https://i.kinja-img.com/gawker-media/image/upl...,2020-07-14T12:30:00Z,"Quest Diagnostics, the largest private medical..."
98,"{'id': None, 'name': 'Gizmodo.com'}","Yessenia Funes on Earther, shared by Brian Kah...",California’s Firefighter Prison Camps go on Lo...,California’s wildland firefighting crew isn’t ...,https://earther.gizmodo.com/california-s-firef...,https://i.kinja-img.com/gawker-media/image/upl...,2020-07-07T13:00:00Z,Californias wildland firefighting crew isnt wh...
